# Local and Central DP with Fed-BioMed: MONAI 2d image registration

## Introduction

This tutorial shows how to deploy in Fed-BioMed the 2d image registration example provided in the project MONAI (https://monai.io/), trained with Differential Privacy (DP). We are going to compare results of:
* non private training
* train with Local Differential Privacy (LDP)
* train with Central Differential Privacy (CDP)

In order to enforce differential privacy during training (both local and central) we will rely on the Opcaus library (https://opacus.ai/). 

## Image Registration

Image registration is the process of transforming and recalibrating different images into one coordinate system. It makes possible to compare several images captured with the same modality.

In this tutorial, we are using a UNet-like registration network ( https://arxiv.org/abs/1711.01666 ).
Goal of the notebook is to train a model given moving images and fixed images (recalibrated images). 

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Creating MedNIST nodes

MedNIST provides an artificial 2d classification dataset created by gathering different medical imaging datasets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.

To proceed with the tutorial, we created an iid partitioning of the MedNIST dataset between 3 clients. Each client has 3000 image samples for each class. The training partitions are availables at the following link:

https://drive.google.com/file/d/1vLIcBdtdAhh6K-vrgCFy_0Y55dxOWZwf/view

The dataset owned by each client has structure:


└── client_*/

    ├── AbdomenCT/
    
    └── BreastMRI/
    
    └── CXR/
    
    └── ChestCT/
    
    └── Hand/
    
    └── HeadCT/   

To create the federated dataset, we follow the standard procedure for node creation/population of Fed-BioMed. 
After activating the fedbiomed network with the commands

`source ./scripts/fedbiomed_environment network`

and 

`./scripts/fedbiomed_run network`

we create a first node by using the commands

`source ./scripts/fedbiomed_environment node`

`./scripts/fedbiomed_run node start`

We then poulate the node with the data of first client:

`./scripts/fedbiomed_run node add`

We select option 3 (images) to add MedNIST partition of client 1, by just picking the folder of client 1. 
Assign tag `mednist` to the data when asked.

We can further check that the data has been added by executing `./scripts/fedbiomed_run node list`

Following the same procedure, we create the other two nodes with the datasets of client 2 and client 3 respectively.


## Running Fed-BioMed Researcher

We are now ready to start the reseracher enviroment with the command `source ./scripts/fedbiomed_environment researcher`, and open the Jupyter notebook. 

We can first quesry the network for the mednist dataset. In this case, the nodes are sharing the respective partitions unsing the same tag `mednist`:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-05-31 11:18:52,193 fedbiomed INFO - Component environment:
2022-05-31 11:18:52,194 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-05-31 11:18:52,507 fedbiomed INFO - Messaging researcher_079823e9-a24e-423e-a02f-9a8b64a18e5a successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x140c0a850>
2022-05-31 11:18:52,550 fedbiomed INFO - Listing available datasets in all nodes... 
2022-05-31 11:19:02,557 fedbiomed INFO - 
 Node: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa | Number of Datasets: 1 
+---------+-------------+--------------------------+-----------------+--------------------+
| name    | data_type   | tags                     | description     | shape              |
+=========+=============+==========================+=================+====================+
| MEDNIST | mednist     | ['#MEDNIST', '#dataset'] | MEDNIST dataset | [58954, 3, 64, 64] |
+---------+-------------+--------------------------+-----------------+---------

{'node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa': [{'name': 'MEDNIST',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MEDNIST dataset',
   'shape': [58954, 3, 64, 64]}],
 'node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f': [{'name': 'MEDNIST',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MEDNIST dataset',
   'shape': [58954, 3, 64, 64]}]}

## Create an experiment to train a model on the data found

The code for network and data loader of the MONAI tutorial can now be deployed in Fed-BioMed.
We first import the necessary modules from `fedbiomed` and `monai` libraries:

In [3]:
from monai.utils import set_determinism, first
from monai.transforms import (
    EnsureChannelFirstD,
    Compose,
    LoadImageD,
    RandRotateD,
    RandZoomD,
    ScaleIntensityRanged,
    EnsureTypeD,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.config import print_config, USE_COMPILED
from monai.networks.nets import GlobalNet
from monai.networks.blocks import Warp
from monai.apps import MedNISTDataset

We can now define the training plan. Note that we use the standard `TorchTrainingPlan` natively provided in Fed-BioMed. We reuse the `MedNISTDataset` data loader defined in the original MONAI tutorial, which is returned by the method `training_data`, which also implements the data parsing from the nodes `dataset_path`. We should also properly define the `training_routine`, following the MONAI tutorial. According to the MONAI tutorial, the model is the `GlobalNet` and the loss is `MSELoss`.

In [4]:
import os
import numpy as np
import torch
from torch.nn import MSELoss
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.logger import logger
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms
from typing import Union, List

#from torch.utils.data import Dataset, DataLoader
import monai
from monai.utils import set_determinism, first
from monai.transforms import (
    EnsureChannelFirstD,
    Compose,
    LoadImageD,
    RandRotateD,
    RandZoomD,
    ScaleIntensityRanged,
    EnsureTypeD,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.config import print_config, USE_COMPILED
from monai.networks.nets import GlobalNet
from monai.networks.blocks import Warp
from monai.apps import MedNISTDataset


# Here we define the model to be used. 
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["import numpy as np",
                "import monai",
                "from torch.nn import MSELoss",
                "from monai.utils import set_determinism, first",
                "from monai.transforms import (EnsureChannelFirstD,Compose,LoadImageD,RandRotateD,RandZoomD,ScaleIntensityRanged,EnsureTypeD,)",
                "from monai.data import DataLoader, Dataset, CacheDataset",
                "from monai.networks.nets import GlobalNet",
                "from monai.config import USE_COMPILED",
                "from monai.networks.blocks import Warp",
                "from monai.apps import MedNISTDataset",
                "from opacus.validators import ModuleValidator"]
        self.add_dependency(deps)
        
        self.model = GlobalNet(
            image_size=(64, 64),
            spatial_dims=2,
            in_channels=2,  # moving and fixed
            num_channel_initial=16,
            depth=3)
        self.image_loss = MSELoss()
        
        if USE_COMPILED:
            self.warp_layer = Warp(3, "border")
        else:
            self.warp_layer = Warp("bilinear", "border")
            
    def make_optimizer(self,lr):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

    def training_data(self, batch_size = 20):
        # Custom torch Dataloader for MedNIST data
        data_path = self.dataset_path
        # The following line is needed if client structure does not contain the "/MedNIST" folder
        MedNISTDataset.dataset_folder_name = ""
        train_data = MedNISTDataset(root_dir=data_path, section="training", download=False, transform=None)
        training_datadict = [
            {"fixed_hand": item["image"], "moving_hand": item["image"]}
            for item in train_data.data if item["label"] == 4  # label 4 is for xray hands
        ]
        train_transforms = Compose(
            [
                LoadImageD(keys=["fixed_hand", "moving_hand"]),
                EnsureChannelFirstD(keys=["fixed_hand", "moving_hand"]),
                ScaleIntensityRanged(keys=["fixed_hand", "moving_hand"],
                                     a_min=0., a_max=255., b_min=0.0, b_max=1.0, clip=True,),
                RandRotateD(keys=["moving_hand"], range_x=np.pi/4, prob=1.0, keep_size=True, mode="bicubic"),
                RandZoomD(keys=["moving_hand"], min_zoom=0.9, max_zoom=1.1,
                          monaiprob=1.0, mode="bicubic", align_corners=False),
                EnsureTypeD(keys=["fixed_hand", "moving_hand"]),
            ]
        )
        train_ds = CacheDataset(data=training_datadict, transform=train_transforms,
                                cache_rate=1.0, num_workers=0)
        dl = self.MednistDataLoader(train_ds)
        
        return DataManager(dl, batch_size=batch_size, shuffle=True, num_workers=0)

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, moving, fixed):
        ddf = self.forward(torch.cat((moving, fixed), dim=1))
        pred_image = self.warp_layer(moving, ddf)
        loss = self.image_loss(pred_image, fixed)
        return loss
    
    class MednistDataLoader(monai.data.Dataset):
        # Custom DataLoader that inherits from monai's Dataset object
        def __init__(self, dataset):
            self.dataset = dataset

        def __len__(self):
            return len(self.dataset)

        def __getitem__(self, idx):
            return (self.dataset[idx]["moving_hand"],
                    self.dataset[idx]["fixed_hand"])

Finally we import the required modules for running any experiment

In [5]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

# Non-private training

We first train our model in a non-private way. We set the model and training parameters. In particular, we are going to perform 2 epochs over 3 rounds for this experiment. Moreover the training is performed on ~26% of the locally available training data. We are also trying to use GPU if available.

In [6]:
model_args = {'use_gpu': True}

training_args = {
    'batch_size': 16, 
    'lr': 1e-5, 
    'epochs': 1, 
    'dry_run': False,  
    #'batch_maxnum':250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

tags =  ['#MEDNIST', '#dataset']
rounds = 1

The experiment can be now defined, by providing the `mednist` tag, and running the local training on nodes with model defined in `model_path`, standard `aggregator` (FedAvg) and `client_selection_strategy` (all nodes used). Federated learning is going to be perfomed through 3 optimization rounds.

In [7]:
exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None
                )

2022-05-31 10:42:26,692 fedbiomed INFO - Searching dataset with data tags: ['#MEDNIST', '#dataset'] for all nodes
05/31/2022 10:42:26:INFO:Searching dataset with data tags: ['#MEDNIST', '#dataset'] for all nodes
2022-05-31 10:42:36,701 fedbiomed INFO - Node selected for training -> node_5d003ecf-cd99-4b95-8789-1b83985d7c82
05/31/2022 10:42:36:INFO:Node selected for training -> node_5d003ecf-cd99-4b95-8789-1b83985d7c82
2022-05-31 10:42:36,704 fedbiomed INFO - Node selected for training -> node_279fb396-94f8-4e1e-b87b-bdc5e34ec544
05/31/2022 10:42:36:INFO:Node selected for training -> node_279fb396-94f8-4e1e-b87b-bdc5e34ec544
2022-05-31 10:42:36,707 fedbiomed INFO - Checking data quality of federated datasets...
05/31/2022 10:42:36:INFO:Checking data quality of federated datasets...
/Users/mlorenzi/miniconda3/envs/fedbiomed-researcher/lib/python3.9/site-packages/monai/networks/blocks/warp.py:65: UserWarning: monai.networks.blocks.Warp: Using PyTorch native grid_sample.
  warnings.warn("m

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the clients



In [8]:
exp.run()

2022-05-31 10:42:43,079 fedbiomed INFO - Sampled nodes in round 0 ['node_5d003ecf-cd99-4b95-8789-1b83985d7c82', 'node_279fb396-94f8-4e1e-b87b-bdc5e34ec544']
05/31/2022 10:42:43:INFO:Sampled nodes in round 0 ['node_5d003ecf-cd99-4b95-8789-1b83985d7c82', 'node_279fb396-94f8-4e1e-b87b-bdc5e34ec544']
2022-05-31 10:42:43,081 fedbiomed INFO - Sending request 
					 To: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_0599e1ae-cbcd-4d79-87d7-cbd8a8aa4e25', 'job_id': 'b54bebd5-7a80-4d48-9b69-36bf6846fd00', 'training_args': {'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 16, 'lr': 1e-05, 'epochs': 1, 'dry_run': False}, 'training': True, 'model_args': {'use_gpu': True}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/05/31/my_model_3babdf24-6ee9-42ab-a96a-9b784f545a55.py', 'params_url': 'http://loc

2022-05-31 10:42:53,497 fedbiomed INFO - WARNING
					 NODE node_279fb396-94f8-4e1e-b87b-bdc5e34ec544
					 MESSAGE: Node training model on CPU, though researcher requested GPU
-----------------------------------------------------------------
05/31/2022 10:42:53:INFO:WARNING
					 NODE node_279fb396-94f8-4e1e-b87b-bdc5e34ec544
					 MESSAGE: Node training model on CPU, though researcher requested GPU
-----------------------------------------------------------------
2022-05-31 10:42:56,320 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 160/7999 (2%) 
 					 Loss: 0.050987 
					 ---------
05/31/2022 10:42:56:INFO:TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 160/7999 (2%) 
 					 Loss: 0.050987 
					 ---------
2022-05-31 10:42:56,334 fedbiomed INFO - TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 160/7999 (2%) 
 					 Loss:

05/31/2022 10:43:22:INFO:TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 1600/7999 (20%) 
 					 Loss: 0.049976 
					 ---------
2022-05-31 10:43:24,865 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 1760/7999 (22%) 
 					 Loss: 0.037815 
					 ---------
05/31/2022 10:43:24:INFO:TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 1760/7999 (22%) 
 					 Loss: 0.037815 
					 ---------
2022-05-31 10:43:25,019 fedbiomed INFO - TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 1760/7999 (22%) 
 					 Loss: 0.043837 
					 ---------
05/31/2022 10:43:25:INFO:TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 1760/7999 (22%) 
 					 Loss: 0.043837 
					 ---------
2022-05-31 10:43:27,627 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5d003ecf-cd9

2022-05-31 10:43:53,393 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 3360/7999 (42%) 
 					 Loss: 0.050510 
					 ---------
05/31/2022 10:43:53:INFO:TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 3360/7999 (42%) 
 					 Loss: 0.050510 
					 ---------
2022-05-31 10:43:53,679 fedbiomed INFO - TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 3360/7999 (42%) 
 					 Loss: 0.039251 
					 ---------
05/31/2022 10:43:53:INFO:TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 3360/7999 (42%) 
 					 Loss: 0.039251 
					 ---------
2022-05-31 10:43:56,157 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 3520/7999 (44%) 
 					 Loss: 0.044802 
					 ---------
05/31/2022 10:43:56:INFO:TRAINING 
					 NODE_ID: node_5d003ecf-cd9

05/31/2022 10:44:21:INFO:TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 4960/7999 (62%) 
 					 Loss: 0.040720 
					 ---------
2022-05-31 10:44:21,847 fedbiomed INFO - TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 4960/7999 (62%) 
 					 Loss: 0.036956 
					 ---------
05/31/2022 10:44:21:INFO:TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 4960/7999 (62%) 
 					 Loss: 0.036956 
					 ---------
2022-05-31 10:44:24,145 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 5120/7999 (64%) 
 					 Loss: 0.041747 
					 ---------
05/31/2022 10:44:24:INFO:TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 5120/7999 (64%) 
 					 Loss: 0.041747 
					 ---------
2022-05-31 10:44:24,612 fedbiomed INFO - TRAINING 
					 NODE_ID: node_279fb396-94f

2022-05-31 10:44:51,211 fedbiomed INFO - TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 6560/7999 (82%) 
 					 Loss: 0.032821 
					 ---------
05/31/2022 10:44:51:INFO:TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 6560/7999 (82%) 
 					 Loss: 0.032821 
					 ---------
2022-05-31 10:44:53,460 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 6720/7999 (84%) 
 					 Loss: 0.037579 
					 ---------
05/31/2022 10:44:53:INFO:TRAINING 
					 NODE_ID: node_5d003ecf-cd99-4b95-8789-1b83985d7c82 
					 Epoch: 1 | Completed: 6720/7999 (84%) 
 					 Loss: 0.037579 
					 ---------
2022-05-31 10:44:53,909 fedbiomed INFO - TRAINING 
					 NODE_ID: node_279fb396-94f8-4e1e-b87b-bdc5e34ec544 
					 Epoch: 1 | Completed: 6720/7999 (84%) 
 					 Loss: 0.029476 
					 ---------
05/31/2022 10:44:53:INFO:TRAINING 
					 NODE_ID: node_279fb396-94f

05/31/2022 10:45:16:INFO:INFO
					 NODE node_279fb396-94f8-4e1e-b87b-bdc5e34ec544
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-05-31 10:45:23,160 fedbiomed INFO - Downloading model params after training on node_5d003ecf-cd99-4b95-8789-1b83985d7c82 - from http://localhost:8844/media/uploads/2022/05/31/node_params_fc57d636-a3a0-460b-8394-380c79811a73.pt
05/31/2022 10:45:23:INFO:Downloading model params after training on node_5d003ecf-cd99-4b95-8789-1b83985d7c82 - from http://localhost:8844/media/uploads/2022/05/31/node_params_fc57d636-a3a0-460b-8394-380c79811a73.pt
2022-05-31 10:45:23,231 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_152f0d3f-fb01-412e-8d53-d5fa5f59f278.pt successful, with status code 200
05/31/2022 10:45:23:DEBUG:upload (HTTP GET request) of file node_params_152f0d3f-fb01-412e-8d53-d5fa5f59f278.pt successful, with status code 200
2022-05-31 10:45:23,244 fedbiomed INFO - Downloadin

1

# Training with DP

## DP parameters

In order to perform DP training (both local and central) we need to provide to the model and training schemes:
* `clip`: defining the maximal L2 norm of gradients
* `sigma`: defining the strenght of Gaussian noise to be added (either to gradients in case of LDP or to the final local model in case of CDP)

## LDP

### Dimensioning the training parameters with LDP

In [8]:
from fedbiomed.researcher.requests import Requests

req = Requests()
query_nodes = req.list()
# min_dataset_size = 1000 #min([xx[i][0]['shape'][0] for i in xx]) #see training data in model
# tot_dataset_size = 1000*len([xx[i][0]['shape'][0] for i in xx]) #sum([xx[i][0]['shape'][0] for i in xx]) #see training data in model

2022-05-31 11:20:20,266 fedbiomed INFO - Listing available datasets in all nodes... 
05/31/2022 11:20:20:INFO:Listing available datasets in all nodes... 


In [9]:
min_dataset_size = min([query_nodes[i][0]['shape'][0] for i in query_nodes]) #see training data in model
tot_dataset_size = sum([query_nodes[i][0]['shape'][0] for i in query_nodes]) #see training data in model

In [10]:
q = training_args['batch_size']/min_dataset_size
sigma = 1.
clip = 1.
delta = .1/min_dataset_size
max_epsilon =1.
max_N = int(1e5)

In [11]:
from fedbiomed.researcher.privacy.rdp_accountant import get_iterations, compute_rdp

N, eps_list = get_iterations(delta, sigma, q, max_epsilon, max_N)

In [12]:
max_rounds = N/(training_args['epochs'])

In [13]:
assert training_args['epochs']*rounds<=max_rounds, 'Number of rounds not compatible with privacy budget'

print(f'The maximal number of FL rounds for ({max_epsilon},{delta})-LDP training is {max_rounds}')
print('The selected number of FL rounds, '+str(rounds)+
      ',implies ('+str(eps_list[training_args['epochs']*rounds-1])+','+str(delta)+',)-LDP')

The maximal number of FL rounds for (1.0,1.6962377446822949e-06)-LDP training is 100000.0
The selected number of FL rounds, 1,implies (0,1.6962377446822949e-06,)-LDP


We are now going to repeat the same training but with private SGD: at each epoch gradients are clipped and perturbed according to the provided privacy parameters.

## Update training parameters for LDP

In order to perform DP-training we should provide an additional argument to training: the dictionalry `'DP_args'` containing necessary parameters for DP. If we want to perform LDP, we should specify: `'type' : 'local'`.

In [13]:
LDP = {'dp_args': {'type' : 'local', 'sigma': sigma, 'clip': clip}}
model_args.update(LDP)

## Declare and run the LDP training

In [14]:
exp_LDP = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None
                )

2022-05-31 11:13:42,829 fedbiomed INFO - Searching dataset with data tags: ['#MEDNIST', '#dataset'] for all nodes
05/31/2022 11:13:42:INFO:Searching dataset with data tags: ['#MEDNIST', '#dataset'] for all nodes
2022-05-31 11:13:52,838 fedbiomed INFO - Node selected for training -> node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa
05/31/2022 11:13:52:INFO:Node selected for training -> node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa
2022-05-31 11:13:52,840 fedbiomed INFO - Node selected for training -> node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f
05/31/2022 11:13:52:INFO:Node selected for training -> node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f
2022-05-31 11:13:52,844 fedbiomed INFO - Checking data quality of federated datasets...
05/31/2022 11:13:52:INFO:Checking data quality of federated datasets...
/Users/mlorenzi/miniconda3/envs/fedbiomed-researcher/lib/python3.9/site-packages/monai/networks/blocks/warp.py:65: UserWarning: monai.networks.blocks.Warp: Using PyTorch native grid_sample.
  warnings.warn("m

In [15]:
exp_LDP.run()

2022-05-31 11:13:56,155 fedbiomed INFO - Sampled nodes in round 0 ['node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa', 'node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f']
05/31/2022 11:13:56:INFO:Sampled nodes in round 0 ['node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa', 'node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f']
2022-05-31 11:13:56,156 fedbiomed INFO - Sending request 
					 To: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_079823e9-a24e-423e-a02f-9a8b64a18e5a', 'job_id': 'fc1e4f6c-6c79-4fa2-af2a-cb70b2246813', 'training_args': {'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 16, 'lr': 1e-05, 'epochs': 1, 'dry_run': False}, 'training': True, 'model_args': {'use_gpu': True, 'dp_args': {'type': 'local', 'sigma': 1.0, 'clip': 1.0}}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/05/31/my_model_947e2075-7

05/31/2022 11:14:06:INFO:INFO
					 NODE node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x14334aa30>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'lr': 1e-05, 'epochs': 1, 'dry_run': False}
-----------------------------------------------------------------
2022-05-31 11:14:06,229 fedbiomed INFO - WARNING
					 NODE node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa
					 MESSAGE: Node training model on CPU, though researcher requested GPU
-----------------------------------------------------------------
05/31/2022 11:14:06:INFO:WARNING
					 NODE node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa
					 MESSAGE: Node training model on CPU, though researcher requested GPU
-----------------------------------------------------------------
2022-05-31 11:14:11,133 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 130/7

05/31/2022 11:14:56:INFO:TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 2200/7999 (20%) 
 					 Loss: 0.071102 
					 ---------
2022-05-31 11:15:01,180 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Epoch: 1 | Completed: 2100/7999 (20%) 
 					 Loss: 0.063828 
					 ---------
05/31/2022 11:15:01:INFO:TRAINING 
					 NODE_ID: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Epoch: 1 | Completed: 2100/7999 (20%) 
 					 Loss: 0.063828 
					 ---------
2022-05-31 11:15:01,475 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 2530/7999 (22%) 
 					 Loss: 0.077781 
					 ---------
05/31/2022 11:15:01:INFO:TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 2530/7999 (22%) 
 					 Loss: 0.077781 
					 ---------
2022-05-31 11:15:05,288 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b3ed9b42-7a9

2022-05-31 11:15:49,117 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 4410/7999 (42%) 
 					 Loss: 0.063492 
					 ---------
05/31/2022 11:15:49:INFO:TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 4410/7999 (42%) 
 					 Loss: 0.063492 
					 ---------
2022-05-31 11:15:51,229 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Epoch: 1 | Completed: 3000/7999 (40%) 
 					 Loss: 0.062015 
					 ---------
05/31/2022 11:15:51:INFO:TRAINING 
					 NODE_ID: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Epoch: 1 | Completed: 3000/7999 (40%) 
 					 Loss: 0.062015 
					 ---------
2022-05-31 11:15:53,977 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 3080/7999 (44%) 
 					 Loss: 0.069436 
					 ---------
05/31/2022 11:15:53:INFO:TRAINING 
					 NODE_ID: node_7dc031f2-6a6

05/31/2022 11:16:41:INFO:TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 3720/7999 (62%) 
 					 Loss: 0.061743 
					 ---------
2022-05-31 11:16:43,907 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Epoch: 1 | Completed: 6900/7999 (60%) 
 					 Loss: 0.071035 
					 ---------
05/31/2022 11:16:43:INFO:TRAINING 
					 NODE_ID: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Epoch: 1 | Completed: 6900/7999 (60%) 
 					 Loss: 0.071035 
					 ---------
2022-05-31 11:16:46,726 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 8640/7999 (64%) 
 					 Loss: 0.060933 
					 ---------
05/31/2022 11:16:46:INFO:TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 8640/7999 (64%) 
 					 Loss: 0.060933 
					 ---------
2022-05-31 11:16:48,464 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b3ed9b42-7a9

2022-05-31 11:17:35,580 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Epoch: 1 | Completed: 6400/7999 (80%) 
 					 Loss: 0.057447 
					 ---------
05/31/2022 11:17:35:INFO:TRAINING 
					 NODE_ID: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Epoch: 1 | Completed: 6400/7999 (80%) 
 					 Loss: 0.057447 
					 ---------
2022-05-31 11:17:38,576 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 6300/7999 (84%) 
 					 Loss: 0.058826 
					 ---------
05/31/2022 11:17:38:INFO:TRAINING 
					 NODE_ID: node_7dc031f2-6a6e-49a7-ae0c-7441a3c2453f 
					 Epoch: 1 | Completed: 6300/7999 (84%) 
 					 Loss: 0.058826 
					 ---------
2022-05-31 11:17:41,304 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b3ed9b42-7a94-409a-b234-f3edeba6bbfa 
					 Epoch: 1 | Completed: 7790/7999 (82%) 
 					 Loss: 0.059116 
					 ---------
05/31/2022 11:17:41:INFO:TRAINING 
					 NODE_ID: node_b3ed9b42-7a9


--------------------
Fed-BioMed researcher stopped due to keyboard interrupt
--------------------


## CDP

### Dimensioning the training parameters with CDP

In [ ]:
import numpy as np
num_clients = len([query_nodes[i][0]['shape'][0] for i in query_nodes])

# Here we use the same parameters as LDP to evaluate the number of rounds, 
# since we are performing record-level DP

q = training_args['batch_size']/min_dataset_size 
sigma = 1.#/(np.sqrt(num_clients)*training_args['batch_size'])
clip = 1.
delta = .1/min_dataset_size
max_epsilon = 10.
max_N = int(1e5)

N, eps_list = get_iterations(delta, sigma, q, max_epsilon, max_N)

In [ ]:
max_rounds = N/(training_args['epochs'])

In [ ]:
assert rounds<=max_rounds, 'Number of rounds not compatible with privacy budget'

print(f'The maximal number of allowed rounds for ({max_epsilon},{delta})-CDP training is {max_rounds}')
print(f'The selected number of training rounds, '+str(rounds)+
      ',implies ('+str(eps_list[rounds-1])+','+str(delta)+',)-CDP')

## Update training parameters for CDP

If we want to perform CDP, we should update the `'DP_args'` dictionary by setting:  `'type' : 'central'`. Otherwise we are going to keep the same privacy parameters.

In [ ]:
CDP = {'dp_args': {'type' : 'central', 'sigma': sigma/np.sqrt(num_clients), 'clip': clip}}
training_args.update(CDP)

## Declare and run the CDP training

In [ ]:
exp_CDP = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None
                )

In [ ]:
exp_CDP.run()

# Testing


We are now going to test and compare locally the three final federated models on an independent testing partition.
The test dataset is available at this link:

https://drive.google.com/file/d/1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD/

In [ ]:
!pip install matplotlib -q
!pip install gdown -q

In [ ]:
import os
import tempfile
import PIL
import torch
import numpy as np
import matplotlib.pyplot as plt
import gdown
import zipfile
import matplotlib.pyplot as plt

print_config()
set_determinism(42)

Download the testing dataset on the local temporary folder.

In [ ]:
import gdown
import zipfile
import tempfile
import os
from fedbiomed.researcher.environ import environ

tmp_dir = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+os.sep)

resource = "https://drive.google.com/uc?id=1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD"
base_dir = tmp_dir.name
test_file = os.path.join(base_dir, "MedNIST_testing.zip")

gdown.download(resource, test_file, quiet=False)

zf = zipfile.ZipFile(test_file)

for file in zf.infolist():
    zf.extract(file, base_dir)
    
data_dir = os.path.join(base_dir, "MedNIST_testing")

We redefine our custom dataloader (defined previously in  the `TrainingPlan`):

In [ ]:
from monai.data import DataLoader, Dataset, CacheDataset
import monai

class MednistDataLoader(monai.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return (self.dataset[idx]["moving_hand"],
                self.dataset[idx]["fixed_hand"])

Create the testing data loader and pairs of moving vs fixed hands:

In [ ]:
# Use a GPU if you have one + enough memory available
#
#use_cuda = torch.cuda.is_available()
#device = torch.device("cuda:0" if use_cuda else "cpu")
device = 'cpu'


# recreate model
model = GlobalNet(
    image_size=(64, 64),
    spatial_dims=2,
    in_channels=2,  # moving and fixed
    num_channel_initial=16,
    depth=3).to(device)

if USE_COMPILED:
    warp_layer = Warp(3, "border").to(device)
else:
    warp_layer = Warp("bilinear", "border").to(device)

MedNISTDataset.dataset_folder_name = ""
test_data = MedNISTDataset(root_dir=data_dir, section="test", download=False, transform=None)
testing_datadict = [
    {"fixed_hand": item["image"], "moving_hand": item["image"]}
    for item in test_data.data if item["label"] == 4  # label 4 is for xray hands
]
test_transforms = Compose(
            [
                LoadImageD(keys=["fixed_hand", "moving_hand"]),
                EnsureChannelFirstD(keys=["fixed_hand", "moving_hand"]),
                ScaleIntensityRanged(keys=["fixed_hand", "moving_hand"],
                                     a_min=0., a_max=255., b_min=0.0, b_max=1.0, clip=True,),
                RandRotateD(keys=["moving_hand"], range_x=np.pi/4, prob=1.0, keep_size=True, mode="bicubic"),
                RandZoomD(keys=["moving_hand"], min_zoom=0.9, max_zoom=1.1, prob=1.0, mode="bicubic", align_corners=False),
                EnsureTypeD(keys=["fixed_hand", "moving_hand"]),
            ]
        )
val_ds = CacheDataset(data=testing_datadict[:1000], transform=test_transforms,
                      cache_rate=1.0, num_workers=0)
val_dl = MednistDataLoader(val_ds)
val_loader = DataLoader(val_dl, batch_size=16, num_workers=0)

To test the federated models we need to create model instances and assign to it the models parameters estimated at the last federated optimization rounds. Then, we generate predictions of the transformation between pairs. In addition, we evaluate the structural similarity index for each model.

In [ ]:
!pip install torchmetrics -q

from torchmetrics.functional import structural_similarity_index_measure

# Non private training
model = exp.model_instance()
model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])

# training with LDP
model_LDP = exp_LDP.model_instance()
model_LDP.load_state_dict(exp_LDP.aggregated_params()[rounds - 1]['params'])

# training with CDP
model_CDP = exp_CDP.model_instance()
model_CDP.load_state_dict(exp_CDP.aggregated_params()[rounds - 1]['params'])

for moving, fixed in val_loader:
    # Non private training
    ddf = model(torch.cat((moving, fixed), dim=1))
    pred_image = warp_layer(moving, ddf)
    
    # training with LDP
    ddf_LDP = model_LDP(torch.cat((moving, fixed), dim=1))
    pred_image_LDP = warp_layer(moving, ddf_LDP)
    
    # training with CDP
    ddf_CDP = model_CDP(torch.cat((moving, fixed), dim=1))
    pred_image_CDP = warp_layer(moving, ddf_CDP)
    
    # ssim predicted vs ground truth
    # Non private training
    SSIM = structural_similarity_index_measure(pred_image, fixed)
    # training with LDP
    SSIM_LDP = structural_similarity_index_measure(pred_image_LDP, fixed)
    # training with CDP
    SSIM_CDP = structural_similarity_index_measure(pred_image_CDP, fixed)
    
    break

fixed_image = fixed.detach().cpu().numpy()[:, 0]
moving_image = moving.detach().cpu().numpy()[:, 0]
pred_image = pred_image.detach().cpu().numpy()[:, 0]
pred_image_LDP = pred_image_LDP.detach().cpu().numpy()[:, 0]
pred_image_CDP = pred_image_CDP.detach().cpu().numpy()[:, 0]

In [ ]:
print('---> Results for non-private training')
print(f'SSIM = {SSIM}')

print('---> Results for training with LDP')
print(f'SSIM = {SSIM_LDP})')

print('---> Results for training with CDP')
print(f'SSIM = {SSIM_CDP})')

Finally, we can print some example of predictions of all models from the testing dataset.

In [ ]:
%matplotlib inline
batch_size = 10
plt.subplots(batch_size, 5, figsize=(12, 25))
for b in range(batch_size):
    # moving image
    plt.subplot(batch_size, 5, b * 5 + 1)
    plt.axis('off')
    plt.title("moving image")
    plt.imshow(moving_image[b], cmap="gray")
    # fixed image
    plt.subplot(batch_size, 5, b * 5 + 2)
    plt.axis('off')
    plt.title("fixed image")
    plt.imshow(fixed_image[b], cmap="gray")
    # warped moving
    plt.subplot(batch_size, 5, b * 5 + 3)
    plt.axis('off')
    plt.title("predicted image")
    plt.imshow(pred_image[b], cmap="gray")
    # warped moving LDP
    plt.subplot(batch_size, 5, b * 5 + 4)
    plt.axis('off')
    plt.title("predicted image (LDP)")
    plt.imshow(pred_image_LDP[b], cmap="gray")
    # warped moving CDP
    plt.subplot(batch_size, 5, b * 5 + 5)
    plt.axis('off')
    plt.title("predicted image (CDP)")
    plt.imshow(pred_image_CDP[b], cmap="gray")
plt.axis('off')
plt.show()